In [ ]:
# google drive
# from google.colab import drive
# drive.mount('/content/drive')
# file address in google drive
file_root = './'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.models import ResNet50_Weights

In [ ]:
def norm_img(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

In [ ]:
def pltshow(img,gray=False):
    plt.figure(figsize=(5,5))
    plt.axis('off')
    if(gray):
        plt.imshow(img,cmap='gray')
    else:
        plt.imshow(img)

In [ ]:
# load and show original image
img = cv2.imread('%sbeagle0.jpg'%file_root)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

# load and show watermark image
wm = cv2.imread('%savatar2.jpg'%file_root)
wm = cv2.cvtColor(wm,cv2.COLOR_BGR2RGB)
wm = cv2.resize(wm,dsize=(800,800),fx=1,fy=1,interpolation=cv2.INTER_LINEAR)

In [ ]:
# add border to image so that it can be divided perfectly with block_size
def addborder(img,block_size=4):
    diff_x = img.shape[0] % block_size
    diff_y = img.shape[1] % block_size
    if (diff_x==0 and diff_y==0):
        return img
    img = cv2.copyMakeBorder(img,
              0,(block_size-diff_x),
              0,(block_size-diff_y),
              cv2.BORDER_REPLICATE)
    return img

In [ ]:
# tranform an image from original image to dct blocks
# param bk: a 2-dim numpy array
# param block_size: int
def dct_img(bk,block_size=4):
    img_dct_blocks_h = bk.shape[0] // block_size
    img_dct_blocks_w = bk.shape[1] // block_size
    img_dct = np.zeros(shape = (bk.shape[0],bk.shape[1]))
    for h in range(img_dct_blocks_h):
        print('\r',h,end='',flush=True)
        for w in range(img_dct_blocks_w):
            a_block = bk[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size]
            img_dct[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size] =\
            cv2.dct(a_block.astype(np.float64))
    return img_dct

In [ ]:
# recover an image from frequency domain to spatial domain
# param bk: a 2-dim numpy array
# param block_size: int
def idct_img(bk,block_size=4):
    img_dct_blocks_h = bk.shape[0] // block_size
    img_dct_blocks_w = bk.shape[1] // block_size
    img_idct = np.zeros(shape = (bk.shape[0],bk.shape[1]))
    for h in range(img_dct_blocks_h):
        for w in range(img_dct_blocks_w):
            a_block = bk[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size]
            img_idct[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size] =\
            cv2.idct(a_block.astype(np.float64))
    return img_idct

In [ ]:
def embed_wm(img,wm,block_size=4,alpha=0.1):
    res = np.zeros(shape=(img.shape))
    for ch in range(3):
        i_ch = img[...,ch]
        w_ch = wm[...,ch]
        f_ch = dct_img(i_ch,block_size=block_size)
        # w_f_ch = dct_img(w_ch,block_size=block_size)
        # f_ch_new = f_ch + alpha*w_f_ch
        f_ch_new = f_ch + alpha*w_ch
        i_ch_new = idct_img(f_ch_new,block_size=block_size)
        res[...,ch] = i_ch_new
    return res

In [ ]:
def extract_wm(wmed_img,img,block_size=4,alpha=0.1):
    wmed_dct = np.zeros_like(wmed_img)
    img_dct = np.zeros_like(img)
    wm = np.zeros_like(img)
    for ch in range(3):  
        wmed_dct[...,ch] = dct_img(wmed_img[...,ch],block_size=block_size)
        img_dct[...,ch] = dct_img(img[...,ch],block_size=block_size)
        wm[...,ch] = (wmed_dct[...,ch]-img_dct[...,ch]) / alpha
    return wm

In [ ]:
# wmed_img = embed_wm(img,wm,alpha=0.01)
# #wmed_img = np.clip(wmed_img,0,255)
# pltshow(norm_img(wmed_img))

In [ ]:
# wm_extract = extract_wm(wmed_img,img,alpha=0.01)
# pltshow(wm_extract)

In [ ]:
alpha=0.2

In [ ]:
# load and show watermark image
wm = cv2.imread('%secnu.jpg'%file_root)
wm = cv2.cvtColor(wm,cv2.COLOR_BGR2RGB)

In [ ]:
img_beagle = cv2.imread('%sbeagle0.jpg'%file_root)
img_beagle = cv2.cvtColor(img_beagle,cv2.COLOR_BGR2RGB)
wm_beagleresize = cv2.resize(wm,dsize = (960,1280))

In [ ]:
wmed_beagle = embed_wm(img_beagle,wm_beagleresize,alpha=alpha)
pltshow(norm_img(wmed_beagle))

In [ ]:
extracted_wm = extract_wm(wmed_beagle,img_beagle)
pltshow(extracted_wm)

In [ ]:
# check classification result for single image
# param model: torch model
# param file_path: image file path
# param transform: torchvision.transforms
def check_classify_path(model,file_path,transform):
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = torch.unsqueeze(transform(img),0).cuda()
    out = (F.softmax(model(img),dim=1))
    print(torch.max(out))
    print(out.argmax())
# check classification result for single image
# param model: torch model
# param img: 3-dim image array
# param transform: torchvision.transforms
def check_classify_array(model,img,transform):
    img = torch.unsqueeze(transform(img),0)
    img = img.type(torch.FloatTensor)
    img = img.cuda()
    out = (F.softmax(model(img),dim=1))
    print(torch.max(out))
    print(out.argmax())
# check classification result for single image
# param model: torch model
# param img: 4-dim image tensor
def check_classify_tensor(model,img):
    out = (F.softmax(model(img),dim=1))
    print(torch.max(out))
    print(out.argmax())

In [ ]:
# image tranformation
T_3 = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize([256,256]),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
T_2 = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize([256,256])
])
T = transforms.ToTensor()

img_tensor =  T(img)
img_tensor = img_tensor.cuda()
wm_tensor = T(wm)
wm_tensor = wm_tensor.cuda()

In [ ]:
import torch_dct as dct

In [ ]:
img_t_dct = torch.stack((dct.dct_2d(img_tensor[0,...]),dct.dct_2d(img_tensor[1,...]),dct.dct_2d(img_tensor[2,...])),dim=0)

In [ ]:
toPIL(img_t_dct).show()

In [ ]:
img_t_idct = torch.stack((dct.idct_2d(img_t_dct[0,...]),dct.idct_2d(img_t_dct[1,...]),dct.idct_2d(img_t_dct[2,...])),dim=0)

In [ ]:
img_tensor[0,0,0]

In [ ]:
toPIL(img_t_idct).show()

In [ ]:
wm_tensor = T(wm_beagleresize)
wm_tensor = wm_tensor.cuda()

In [ ]:
alpha = 0.1

In [ ]:
img_t_dct_wm = img_t_dct + alpha*wm_tensor

In [ ]:
img_t_idct_wm = torch.stack((dct.idct_2d(img_t_dct_wm[0,...]),dct.idct_2d(img_t_dct_wm[1,...]),dct.idct_2d(img_t_dct_wm[2,...])),dim=0)

In [ ]:
toPIL = transforms.ToPILImage()

In [ ]:
def dct_tensor(img):
    return torch.stack((dct.dct_2d(img[0,...]),dct.dct_2d(img[1,...]),dct.dct_2d(img[2,...])),dim=0)

def idct_tensor(img):
    return torch.stack((dct.idct_2d(img[0,...]),dct.idct_2d(img[1,...]),dct.idct_2d(img[2,...])),dim=0)

In [ ]:
wmed_img = img_t_idct_wm.clip(0,1)

In [ ]:
wmed_dct = dct_tensor(wmed_img)

In [ ]:
diff = (wmed_dct-img_t_dct)/alpha

In [ ]:
toPIL(diff).show()

In [ ]:
toPIL(img_t_idct_wm).show()

In [ ]:
%%time
i = cv2.dct(img[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size,0].astype(np.float64))

In [ ]:
%%time
i = dct.dct_2d(img_tensor[0,h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size])

In [ ]:
i0 = dct.dct_2d(img_tensor[2,...])
toPIL(i0).show()

In [ ]:
block_size = 4
img_dct = torch.tensor(np.zeros((img.shape[2],img.shape[0],img.shape[1])))
img_dct = img_dct.cuda()
for ch in range(3):
    bk = img_tensor[ch,...]
    img_dct_blocks_h = bk.shape[0] // block_size
    img_dct_blocks_w = bk.shape[1] // block_size
    bk_dct = torch.tensor(np.zeros_like(img[...,ch]))
    bk_dct = bk_dct.cuda()
    for h in range(img_dct_blocks_h):
        print('\r',h,end='',flush=True)
        for w in range(img_dct_blocks_w):
            a_block = bk[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size]
            bk_dct[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size] =\
            dct.dct_2d(a_block)
    img_dct[ch,...] = bk_dct
toPIL(img_dct).show()

In [ ]:
block_size = 4
img_idct = torch.tensor(np.zeros((img.shape[2],img.shape[0],img.shape[1])))
img_idct = img_dct.cuda()
for ch in range(3):
    bk = img_dct[ch,...]
    img_dct_blocks_h = bk.shape[0] // block_size
    img_dct_blocks_w = bk.shape[1] // block_size
    bk_idct = torch.tensor(np.zeros_like(img[...,ch]))
    bk_idct = bk_dct.cuda()
    for h in range(img_dct_blocks_h):
        print('\r',h,end='',flush=True)
        for w in range(img_dct_blocks_w):
            a_block = bk[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size]
            bk_idct[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size] =\
            dct.idct_2d(a_block)
    img_idct[ch,...] = bk_idct
toPIL(img_dct).show()

In [ ]:
a = toPIL(wm)

In [ ]:
a

In [ ]:
a_block[0,0]

In [ ]:
def dct_img_tensor(bk,block_size=4):
    img_dct_blocks_h = bk.shape[0] // block_size
    img_dct_blocks_w = bk.shape[1] // block_size
    img_dct = np.zeros(shape = (bk.shape[0],bk.shape[1]))
    for h in range(img_dct_blocks_h):
        for w in range(img_dct_blocks_w):
            a_block = bk[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size]
            img_dct[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size] =\
            cv2.dct(a_block.astype(np.float64))
    return img_dct

In [ ]:
toPIL(img_tensor).show()

In [ ]:
# convert tensor to PILImage (for presentation)
from torchvision.transforms import ToPILImage
show = ToPILImage()

In [ ]:
from torch import Tensor
from torchvision.models.resnet import ResNet
from torch import nn
from typing import Optional, Any, List, Callable, Type, Union
from torchvision.models.resnet import BasicBlock,Bottleneck
class ResNet_with_trans(ResNet):
    def forward(self, x: Tensor) -> Tensor:
        x = transforms.Resize([256,256])(x)
        x = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(x)
        return self._forward_impl(x)

from torchvision.models.resnet import ResNet50_Weights
from torchvision.models._utils import _ovewrite_named_param
def resnet50_with_trans(*, weights: Optional[ResNet50_Weights] = None, progress: bool = True, **kwargs: Any) -> ResNet:
    weights = ResNet50_Weights.verify(weights)
    # model = _resnet(Bottleneck, [3, 4, 6, 3], weights, progress, **kwargs)
    block = Bottleneck
    layers = [3, 4, 6, 3]

    if weights is not None:
        _ovewrite_named_param(kwargs, "num_classes", len(weights.meta["categories"]))

    model = ResNet_with_trans(block, layers, **kwargs)

    if weights is not None:
        model.load_state_dict(weights.get_state_dict(progress=progress))
    
    return model

In [ ]:
import torchattacks

In [ ]:
def attack_and_check(model,T,atk,img,target):
    origin = torch.unsqueeze(T(img),0)
    origin = origin.type(torch.FloatTensor)
    origin = origin.cuda()
    adv_images = atk(origin, target)
    out_per = (F.softmax(model(adv_images),dim=1))
    print(torch.max(out_per))
    print(out_per.argmax())
    adv_image = show(adv_images.cpu().detach()[0])
    # adv_image.show()
    return np.array(adv_image)

In [ ]:
resnet = resnet50_with_trans(weights=ResNet50_Weights.IMAGENET1K_V1)
resnet = resnet.cuda()
resnet = resnet.eval()

In [ ]:
wmed_beagle = np.clip(wmed_beagle,0,255)
pltshow(norm_img(wmed_beagle))

In [ ]:
ex_ = extract_wm(wmed_beagle,img_beagle,alpha=0.1)
pltshow(norm_img(ex_))

In [ ]:
wmed_beagle = embed_wm(img_beagle,wm_beagleresize,alpha=alpha)
wmed_beagle = np.clip(wmed_beagle,0,225)
wmed_beagle = norm_img(wmed_beagle)
pltshow(wmed_beagle)

In [ ]:
check_classify_array(resnet,wmed_beagle,T)

In [ ]:
target = torch.tensor([162]).cuda()
cw_atk = torchattacks.CW(resnet, c=1, kappa=0, steps=100, lr=0.01)
cw_beagle = attack_and_check(resnet,T,cw_atk,wmed_beagle,target)
pltshow(cw_beagle)

In [ ]:
target = torch.tensor([162]).cuda()
gamma = 1/255
pgd_atk = torchattacksattack = torchattacks.PGD(resnet, eps=8/255, alpha=gamma, steps=40, random_start=True)
pgd_beagle = attack_and_check(resnet,T,pgd_atk,wmed_beagle,target)
pltshow(pgd_beagle)

In [ ]:
per = (pgd_beagle-wmed_beagle)/gamma

per_dct = np.stack((dct_img(per[...,0]),
                    dct_img(per[...,1]),
                    dct_img(per[...,2])),
                   axis=2)

wm_per = per_dct * (gamma/alpha)

wm_perturbed = wm_per+wm_beagleresize

wm_perturbed = np.clip(wm_perturbed,0,225)
pltshow(norm_img(wm_perturbed))

In [ ]:
wmpered_beagle = embed_wm(img_beagle,wm_perturbed,alpha=alpha)
wmpered_beagle = np.clip(wmpered_beagle,0,225)
wmpered_beagle = norm_img(wmpered_beagle)
pltshow(wmed_beagle)

In [ ]:
check_classify_array(resnet,wmpered_beagle,T)

In [ ]:
def attack_and_check(model,T,atk,img,target):
  origin = torch.unsqueeze(T(img),0).cuda()
  adv_images = atk(origin, target)
  out_per = (F.softmax(model(adv_images),dim=1))
  print(torch.max(out_per))
  print(out_per.argmax())
  adv_image = show(adv_images.cpu().detach()[0])
  # adv_image.show()
  return np.array(adv_image)